In [1]:
# ======================== Full Corrected Code ======================== #

# Install CatBoost
!pip install catboost

# Import Libraries
import os
import cv2
import numpy as np
import pandas as pd
import torch
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch.nn as nn
import torch.optim as optim

# ======================== STEP 1: Mount Google Drive ======================== #
from google.colab import drive
drive.mount('/content/drive')

# ======================== STEP 2: Set Folder Paths ======================== #
mri_normal_path = "/content/drive/MyDrive/Datasets/Parkinson's MRI/normal"
mri_parkinsons_path = "/content/drive/MyDrive/Datasets/Parkinson's MRI/parkinson"
gait_csv_path = "/content/drive/MyDrive/Datasets/parkinsons_extracted_features_freq.csv"
voice_file = "/content/drive/MyDrive/Datasets/Parkinsons_voice/voice.data"

# ======================== STEP 3: Load Models ======================== #
mri_model = joblib.load("/content/drive/MyDrive/ParkinsonsModels/mri_model.pkl")
gait_model = joblib.load("/content/drive/MyDrive/ParkinsonsModels/gait_model.pkl")
voice_model = joblib.load("/content/drive/MyDrive/ParkinsonsModels/voice_model.pkl")

# ======================== STEP 4: Load MRI Images ======================== #
def load_mri_images(folder_path, label, image_size=(128, 128)):
    data = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_resized = cv2.resize(img, image_size)
            img_flat = img_resized.flatten()
            data.append(img_flat)
            labels.append(label)
    return np.array(data), np.array(labels)

X_normal, y_normal = load_mri_images(mri_normal_path, label=0)
X_parkinsons, y_parkinsons = load_mri_images(mri_parkinsons_path, label=1)
X_mri = np.vstack((X_normal, X_parkinsons))
y_mri = np.concatenate((y_normal, y_parkinsons))

# ======================== STEP 5: Load Gait Features ======================== #
gait_df = pd.read_csv(gait_csv_path)
X_gait = gait_df.drop(columns=["label", "filename"]).values
y_gait = gait_df["label"].values

# ======================== STEP 6: Load Voice Features ======================== #
voice_df = pd.read_csv(voice_file)
voice_df.drop("name", axis=1, inplace=True)
X_voice = voice_df.drop("status", axis=1).values
y_voice = voice_df["status"].values

# ======================== STEP 7: Split Each Dataset ======================== #
X_mri_train, X_mri_test, y_mri_train, y_mri_test = train_test_split(X_mri, y_mri, test_size=0.2, stratify=y_mri, random_state=42)
X_gait_train, X_gait_test, y_gait_train, y_gait_test = train_test_split(X_gait, y_gait, test_size=0.2, stratify=y_gait, random_state=42)
X_voice_train_raw, X_voice_test_raw, y_voice_train, y_voice_test = train_test_split(X_voice, y_voice, test_size=0.2, stratify=y_voice, random_state=42)

# ======================== STEP 8: Process Gait Data ======================== #
gait_scaler = StandardScaler()
X_gait_train_scaled = gait_scaler.fit_transform(X_gait_train)
X_gait_test_scaled = gait_scaler.transform(X_gait_test)
pred_gait_probs = gait_model.predict_proba(X_gait_test_scaled)[:, 1]

# ======================== STEP 9: Process Voice Data ======================== #
# Load preprocessing objects
voice_scaler = joblib.load('/content/drive/MyDrive/Datasets/Parkinsons_voice/voice_scaler.pkl')
voice_poly = joblib.load('/content/drive/MyDrive/Datasets/Parkinsons_voice/voice_poly.pkl')
voice_rfe = joblib.load('/content/drive/MyDrive/Datasets/Parkinsons_voice/voice_rfe.pkl')

# Apply preprocessing
X_voice_test_scaled = voice_scaler.transform(X_voice_test_raw)
X_voice_test_poly = voice_poly.transform(X_voice_test_scaled)
X_voice_test_selected = voice_rfe.transform(X_voice_test_poly)

# Predict probabilities using the stacked model
pred_voice_probs = voice_model.predict_proba(X_voice_test_selected)[:, 1]

# ======================== STEP 10: Process MRI Data ======================== #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mri_model = mri_model.to(device)

X_mri_test_tensor = torch.tensor(X_mri_test).float().view(-1, 1, 128, 128)
X_mri_test_tensor_rgb = X_mri_test_tensor.repeat(1, 3, 1, 1).to(device)

with torch.no_grad():
    mri_outputs = mri_model(X_mri_test_tensor_rgb)
    pred_mri_probs = torch.sigmoid(mri_outputs).cpu().numpy().flatten()

# ======================== STEP 11: Align Test Sizes ======================== #
min_size = min(len(pred_mri_probs), len(pred_gait_probs), len(pred_voice_probs))

# Trim probabilities
pred_mri_probs = pred_mri_probs[:min_size]
pred_gait_probs = pred_gait_probs[:min_size]
pred_voice_probs = pred_voice_probs[:min_size]

# Get class predictions and trim
pred_mri_labels = (pred_mri_probs > 0.5).astype(int)
pred_gait_labels = gait_model.predict(X_gait_test_scaled)[:min_size]
pred_voice_labels = voice_model.predict(X_voice_test_selected)[:min_size]
# Create extended meta-feature set
X_meta = np.vstack((
    pred_mri_probs, pred_gait_probs, pred_voice_probs,  # Probabilities
    pred_mri_labels, pred_gait_labels, pred_voice_labels  # Predicted Classes
)).T

# Align labels
y_meta = y_mri_test[:min_size]

# ======================== STEP 12: Hyperparameter Tuning ======================== #
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000]
}

grid = GridSearchCV(LogisticRegression(), param_grid, cv=3)
grid.fit(X_meta, y_meta)

best_log_reg = grid.best_estimator_
print("✅ Best Hyperparameters:", grid.best_params_)

# ======================== STEP 13: LOOCV Evaluation ======================== #
loo = LeaveOneOut()
y_true_lr = []
y_pred_lr = []

for train_idx, test_idx in loo.split(X_meta):
    X_train, X_test = X_meta[train_idx], X_meta[test_idx]
    y_train, y_test = y_meta[train_idx], y_meta[test_idx]

    best_log_reg.fit(X_train, y_train)
    pred = best_log_reg.predict(X_test)

    y_true_lr.append(y_test[0])
    y_pred_lr.append(pred[0])

# ======================== STEP 14: Final Results ======================== #
accuracy_lr = accuracy_score(y_true_lr, y_pred_lr) * 100
print("\n✅ Tuned Logistic Regression Meta-Classifier LOOCV Accuracy: {:.2f}%".format(accuracy_lr))
print("\nClassification Report:\n", classification_report(y_true_lr, y_pred_lr))
print("\nConfusion Matrix:\n", confusion_matrix(y_true_lr, y_pred_lr))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00
Mounted at /content/drive


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

Mounted at /content/drive
Checking MRI Model at /content/drive/MyDrive/ParkinsonsModels/mri_model.pkl
✅ Loaded model successfully: /content/drive/MyDrive/ParkinsonsModels/mri_model.pkl
✅ MRI Model loaded successfully.

Checking GAIT Model at /content/drive/MyDrive/ParkinsonsModels/gait_model.pkl
✅ Loaded model successfully: /content/drive/MyDrive/ParkinsonsModels/gait_model.pkl
✅ GAIT Model loaded successfully.

Checking Voice Model at /content/drive/MyDrive/ParkinsonsModels/voice_model.pkl
❌ Failed to load model: /content/drive/MyDrive/ParkinsonsModels/voice_model.pkl
Error: No module named 'catboost'
❌ Voice Model failed to load.

